In [1]:
!pip install -Uqqq pip
!pip install -qqq langchain==0.1.5
!pip install -qqq openai==1.11.1
!pip install -qqq watermark==2.4.3
# !pip install -Uqqq chromadb==0.4.22 
!pip install -qqq tiktoken==0.5.2 
!pip install -qqq langchain-openai==0.0.5
!pip install -qqq huggingface-hub==0.20.3
!pip install -qqq sentence-transformers

# !pip install -Uqqq pip --progress-bar off
# !pip install -qqq langchain==0.0.148 --progress-bar off
# !pip install -qqq openai==0.27.4 --progress-bar off
# !pip install -qqq tiktoken==0.3.3 --progress-bar off
# !pip install -qqq watermark==2.3.1 --progress-bar off
# !pip install -qqq huggingface-hub==0.14.0 --progress-bar off
# !pip install -qqq sentence_transformers==2.2.2 --progress-bar off

In [14]:
import os
import textwrap
from getpass import getpass

import huggingface_hub
import langchain
import openai
from langchain import HuggingFaceHub, LLMChain, PromptTemplate
#from langchain.callbacks.base import CallbackManager
from langchain.callbacks.manager import AsyncCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from dotenv import load_dotenv

In [4]:
%load_ext watermark
%watermark --iversions -v -m

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.21.0

Compiler    : Clang 14.0.0 (clang-1400.0.29.202)
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

huggingface_hub: 0.20.3
openai         : 1.11.1
langchain      : 0.1.5



In [6]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [5]:
# OPENAI_API_KEY = getpass()

··········


In [6]:
# HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [9]:
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
load_dotenv()

True

## Large Language Models

In [22]:
template = """Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

question = (
#    "What is the relationship between Jim and Dwight from the TV show The Office?"
    "Write a nutrition plan for pregnant woman, including all week days."
)

print(prompt.format(question=question))

Question: Write a nutrition plan for pregnant woman, including all week days.

Answer:


In [23]:
repo_id = "declare-lab/flan-alpaca-large"
# repo_id = "declare-lab/Flan-Alpaca-XL"
flan = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0, "max_length": 128}
)

response = flan.invoke(prompt.format(question=question))
print_response(response)

A nutrition plan for pregnant woman should include a balanced diet that includes a variety of fruits


In [25]:
text_davinci_003 = OpenAI(temperature=0)
response = text_davinci_003(prompt.format(question=question))
print_response(response.strip())

Monday: Breakfast: - Whole grain toast with avocado spread - Scrambled eggs - Fresh fruit (such as
berries or oranges) - Glass of milk  Snack: - Greek yogurt with granola and sliced almonds  Lunch: -
Grilled chicken salad with mixed greens, tomatoes, cucumbers, and a light vinaigrette dressing -
Whole grain crackers with hummus - Glass of water  Snack: - Apple slices with peanut butter  Dinner:
- Baked salmon with quinoa and roasted vegetables (such as broccoli, carrots, and bell peppers) -
Glass of water  Tuesday: Breakfast: - Oatmeal with almond milk, topped with sliced bananas and
walnuts - Hard boiled egg - Glass of orange juice  Snack: - Carrot sticks with ranch dip  Lunch: -
Whole grain wrap with turkey, avocado, lettuce, and tomato - Side of fruit salad - Glass of water
Snack: - Whole grain crackers with cheese slices  Dinner: - Grilled lean beef with sweet potato and
steamed green beans - Glass of water  Wednesday: Breakfast: - Whole grain waffles with Greek yogurt
and fresh be

### Multiple Completions

In [26]:
text_davinci_003 = OpenAI(temperature=0.4, n=3, best_of=3)
response = text_davinci_003.generate([prompt.format(question=question)])
response

LLMResult(generations=[[Generation(text='\n\nMonday:\nBreakfast:\n- Whole grain toast with avocado spread and scrambled eggs\n- Fresh fruit salad (kiwi, berries, oranges)\n- Glass of milk or fortified orange juice\n\nSnack:\n- Greek yogurt with honey and chopped nuts\n\nLunch:\n- Grilled chicken breast with quinoa and roasted vegetables (bell peppers, zucchini, carrots)\n- Small side salad with balsamic vinaigrette dressing\n\nSnack:\n- Apple slices with almond butter\n\nDinner:\n- Baked salmon with brown rice and steamed broccoli\n- Glass of water with lemon\n\nTuesday:\nBreakfast:\n- Oatmeal with almond milk, topped with sliced bananas and chia seeds\n- Hard boiled egg\n- Glass of orange juice\n\nSnack:\n- Hummus with whole wheat pita and sliced cucumbers\n\nLunch:\n- Whole wheat wrap filled with grilled vegetables, black beans, and avocado\n- Side of fresh fruit (grapes, pineapple)\n\nSnack:\n- Carrot sticks with ranch dip\n\nDinner:\n- Turkey meatloaf with sweet potato mash and ste

In [27]:
for i, generation in enumerate(response.generations[0]):
    print(f"Generation {i + 1}\n")
    print_response(generation.text.strip())
    print()

Generation 1

Monday: Breakfast: - Whole grain toast with avocado spread and scrambled eggs - Fresh fruit salad
(kiwi, berries, oranges) - Glass of milk or fortified orange juice  Snack: - Greek yogurt with honey
and chopped nuts  Lunch: - Grilled chicken breast with quinoa and roasted vegetables (bell peppers,
zucchini, carrots) - Small side salad with balsamic vinaigrette dressing  Snack: - Apple slices with
almond butter  Dinner: - Baked salmon with brown rice and steamed broccoli - Glass of water with
lemon  Tuesday: Breakfast: - Oatmeal with almond milk, topped with sliced bananas and chia seeds -
Hard boiled egg - Glass of orange juice  Snack: - Hummus with whole wheat pita and sliced cucumbers
Lunch: - Whole wheat wrap filled with grilled vegetables, black beans, and avocado - Side of fresh
fruit (grapes, pineapple)  Snack: - Carrot sticks with ranch dip  Dinner: - Turkey meatloaf with
sweet potato mash and steamed green beans - Glass of water with lemon  Wednesday: Breakfast: -

## Embeddings

In [28]:
text = """
At the start of The Office, it was apparent that Dwight and Jim rarely got along. 
They were too competitive and too different of people to get along. 
But as the show goes on, time proves to be the one thing that holds Dwight and Jim together. 
They had been through too much as coworkers and friends to not give in to a friendship.

Aside from the toxic (albeit hilarious) relationship between Michael and Toby, 
Dwight and Jim come in second place with the most destructive relationship. 
From pranking each other to being a part of each other's weddings, 
Jim and Dwight have been through the wringer as friends.

The first season only touched the surface of Dwight and Jim’s friendship. 
Viewers saw how cutthroat they could be when Dwight stole Jim’s biggest sale of 
the year in ‘Diversity Day.’ Jim got revenge on Dwight, of course, in ‘Healthcare,’ 
when he made up various diseases that he and Pam had when Dwight asked for 
everyone’s medical history. By the end of the short season, Dwight shot his shot 
with Katy—the purse girl—but Jim was the one who ended up taking her home.

Looking at the ups and downs of Dwight and Jim’s friendship, it’s surprising 
that Jim invited Dwight to his house party in ‘Email Survalience.’ But things 
get back to normal when Jim gave Dwight bad advice when it was announced that 
Dwight had to speak at a salesman convention. The advice Jim gave Dwight was in 
jest but it ended up working for Dwight in the end.
"""

source: https://screenrant.com/jim-halpert-dwight-schrute-friendship-timeline-the-office/#season-1-cutthroat-friends

In [29]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [30]:
embedding = hf_embeddings.embed_query(text)

In [31]:
len(embedding)

384

In [32]:
embedding[:5]

[-0.1361340433359146,
 0.03486044332385063,
 0.050754666328430176,
 -0.06413129717111588,
 -0.05491666495800018]

In [33]:
openai_embeddings = OpenAIEmbeddings()

In [34]:
embedding = openai_embeddings.embed_query(text)

In [35]:
len(embedding)

1536

In [46]:
embedding[:5]

[-0.01814699370231159,
 -0.013020798642834074,
 0.024346111489793782,
 -0.033644788171017063,
 0.0029091483623929935]

## Chat Models

In [37]:
chat_gpt = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

question = (
    "What is the relationship between Jim and Dwight from the TV show The Office?"
)

response = chat_gpt.invoke([HumanMessage(content=question)])
response

AIMessage(content="Jim and Dwight have a complex relationship throughout the TV show The Office. Initially, they are portrayed as rivals, with Jim often pranking Dwight and finding amusement in his eccentricities. Dwight, on the other hand, is constantly trying to outdo Jim and prove himself as the superior employee.\n\nHowever, as the series progresses, their relationship evolves. They develop a begrudging respect for each other and occasionally work together on various projects. Jim also becomes more protective of Dwight, particularly when it comes to their shared workplace and colleagues.\n\nDespite their differences, Jim and Dwight also share moments of genuine friendship and understanding. They occasionally bond over shared interests, and Dwight even acts as Jim's best man at his wedding. Overall, their relationship is a mix of rivalry, friendship, and mutual respect, making them one of the most iconic duos in the show.")

In [38]:
print_response(response.content)

Jim and Dwight have a complex relationship throughout the TV show The Office. Initially, they are
portrayed as rivals, with Jim often pranking Dwight and finding amusement in his eccentricities.
Dwight, on the other hand, is constantly trying to outdo Jim and prove himself as the superior
employee.  However, as the series progresses, their relationship evolves. They develop a begrudging
respect for each other and occasionally work together on various projects. Jim also becomes more
protective of Dwight, particularly when it comes to their shared workplace and colleagues.  Despite
their differences, Jim and Dwight also share moments of genuine friendship and understanding. They
occasionally bond over shared interests, and Dwight even acts as Jim's best man at his wedding.
Overall, their relationship is a mix of rivalry, friendship, and mutual respect, making them one of
the most iconic duos in the show.


In [39]:
messages = [
    SystemMessage(content="You're an expert on the TV show The Office."),
    HumanMessage(content="What is the relationship between Jim and Dwight?"),
]
response = chat_gpt(messages)
print_response(response.content)

The relationship between Jim Halpert and Dwight Schrute on The Office is a complex one that evolves
throughout the series. Initially, Jim and Dwight have a contentious relationship, with Jim often
pranking and teasing Dwight, who takes his job very seriously as the Assistant to the Regional
Manager. These pranks, such as putting Dwight's stapler in Jello or impersonating him, become a
recurring theme throughout the show.  However, as the series progresses, Jim and Dwight's
relationship becomes more nuanced. They develop a sort of begrudging respect for each other, and
occasionally even team up against common enemies or for mutual benefit. They have moments of
camaraderie and understanding, particularly when they find themselves in unusual situations or
facing challenges together.  Despite their differences, Jim and Dwight also share moments of genuine
friendship and support. Jim is often there to offer advice or encouragement to Dwight, and Dwight
occasionally shows a softer side towar

### Use Prompt Templates

In [40]:
question = "What is the relationship between Jim and Dwight?"

In [41]:
system_template = "You're an expert on the TV show The Office. You {style}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)
chat_prompt.format_prompt(
    style="reply in thoughtful and philosophical manner", question=question
).to_messages()

[SystemMessage(content="You're an expert on the TV show The Office. You reply in thoughtful and philosophical manner."),
 HumanMessage(content='What is the relationship between Jim and Dwight?')]

In [42]:
messages = chat_prompt.format_prompt(
    style="reply in thoughtful and philosophical manner", question=question
).to_messages()
response = chat_gpt(messages)
print_response(response.content)

The relationship between Jim and Dwight on The Office is a fascinating exploration of the
complexities of human connection. At first glance, one might perceive them as mere adversaries,
constantly engaged in a battle of wits and pranks. However, upon closer examination, their dynamic
reveals a deeper and more nuanced bond.  Jim and Dwight's relationship can be seen as a reflection
of the yin and yang concept, where two seemingly opposing forces find balance and harmony within
each other. They are two sides of the same coin, representing contrasting personalities and
approaches to life. Jim, with his laid-back and playful nature, serves as a foil to Dwight's intense
and rule-oriented demeanor.  Despite their differences, there is an undeniable mutual respect that
underlies their interactions. They push each other to be better versions of themselves, challenging
each other's beliefs and pushing the boundaries of their comfort zones. In a way, they serve as
mirrors, reflecting back the st

In [43]:
messages = chat_prompt.format_prompt(
    style="reply in sarcastic and outrageous manner", question=question
).to_messages()
response = chat_gpt(messages)
print_response(response.content)

Oh, Jim and Dwight? They're practically soulmates! They have this beautiful, unbreakable bond built
on mutual love and respect. I mean, who wouldn't want to be best friends with someone who constantly
pranks them and steals their stapler? It's a match made in heaven, really.


### Streaming

In [45]:
streaming_chat_gpt = ChatOpenAI(
    streaming=True,
    callback_manager=AsyncCallbackManager([StreamingStdOutCallbackHandler()]),
    temperature=0,
    verbose=True,
)

In [46]:
messages = chat_prompt.format_prompt(
    style="reply in sarcastic and outrageous manner", question=question
).to_messages()
response = streaming_chat_gpt(messages)

Oh, Jim and Dwight? They're practically soulmates! They have this beautiful, unbreakable bond built on mutual love and respect. I mean, who wouldn't want to be best friends with someone who constantly pranks them and steals their stapler? It's a match made in heaven, really.